In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import random
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers
from tensorflow.keras import Model
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tensorflow import keras
from tqdm import tqdm


print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

def seed_tensorflow(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

2024-07-28 22:04:18.217853: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-28 22:04:18.243082: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 22:04:18.243116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 22:04:18.243740: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 22:04:18.247949: I tensorflow/core/platform/cpu_feature_guar

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-07-28 22:04:19.380554: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:19.406973: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:19.407146: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
select_label = 'effective_play'
CPF_model = "./model/kuairand_CPF/Autoint_{}.h5".format(select_label)

In [3]:
para = {'data_path':'./kuai_input/',
        'embedding_dim':64,
        'seed':0,
        'lr':1e-5,
        'batch_size':512,
        'epochs':64,
        'verbose':1,
        'callback':{
                     'monitor':'val_auc',
                     'patience':10,
                     'base_model':CPF_model
                     },
        'mlp_dims':[256,128,64],
        'mlp_act':'relu',
        'mlp_dps':[.5,.5,.5],
       }

In [4]:
action = pd.read_csv(para['data_path']+"action.csv")
feed_emb = np.load(para['data_path']+"embeddings.npy")

In [5]:
action

,user_id,video_id,date,hourmin,time_ms,is_click,is_like,is_follow,is_comment,is_forward,...,outsite_share_all_cnt,pcr,duration_level,threshold,binary_train_label,short_play,effective_play,long_play,complete_play,test_label
0,0,1399,20220411,1900,1649675512388,0,0,0,0,0,...,3.902597,0.006598,4,0.335938,0,1,0,0,0,0
1,0,6857,20220416,2000,1650111976017,0,0,0,0,0,...,6.041237,0.000000,2,0.421875,0,1,0,0,0,0
2,0,5590,20220420,1600,1650444367095,0,0,0,0,0,...,4.868750,0.008224,4,0.335938,0,1,0,0,0,0
3,1,5899,20220411,1100,1649645295928,0,0,0,0,0,...,0.805882,0.000000,4,0.335938,0,1,0,0,0,0
4,1,3376,20220411,1100,1649648827559,0,0,0,0,0,...,154.725146,0.024707,2,0.421875,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1266970,25491,1362,20220502,1100,1651461216855,0,0,0,0,0,...,11.206897,0.010102,3,0.386719,0,1,0,0,0,0
1266971,25491,5690,20220502,1600,1651478750711,0,0,0,0,0,...,35.254658,0.064382,1,0.492188,0,1,0,0,0,0
1266972,25491,5690,20220502,1600,1651478750711,0,0,0,0,0,...,35.254658,0.064382,1,0.492188,0,1,0,0,0,0
1266973,25491,3014,20220507,1800,1651918814747,1,0,0,0,0,...,1.873563,1.000000,1,0.492188,1,0,1,1,1,0


In [6]:
action = action[['date','user_id','video_id','duration_level','tab','user_active_degree','author_id','music_id',
                 'short_play', 'binary_train_label', 'effective_play',
                 'long_play', 'complete_play']]
categories_to_keep = ['full_active', 'high_active', 'middle_active','low_active']

action = action[action['user_active_degree'].isin(categories_to_keep)]

category_mapping = {
    'low_active': 0,
    'middle_active': 1,
    'high_active': 2,
    'full_active': 3
}

action['user_active_degree_encoded'] = action['user_active_degree'].map(category_mapping)
action['music_id_encoded'], _ = pd.factorize(action['music_id'])

In [7]:
train= action.groupby('user_id').apply(lambda x: x[:int(len(x)*0.6)]).reset_index(drop=True)
valid= action.groupby('user_id').apply(lambda x: x[int(len(x)*0.6):int(len(x)*0.8)]).reset_index(drop=True)
test= action.groupby('user_id').apply(lambda x: x[int(len(x)*0.8):]).reset_index(drop=True)

In [8]:
train

,date,user_id,video_id,duration_level,tab,user_active_degree,author_id,music_id,short_play,binary_train_label,effective_play,long_play,complete_play,user_active_degree_encoded,music_id_encoded
0,20220411,0,1399,4,1,full_active,5799,9154013497,1,0,0,0,0,3,0
1,20220416,0,6857,2,0,full_active,5412,9145692691,1,0,0,0,0,3,1
2,20220420,0,5590,4,1,full_active,4571,9156647291,1,0,0,0,0,3,2
3,20220411,0,6606,4,1,full_active,2881,9141209460,1,0,0,0,0,3,643
4,20220411,0,6606,4,0,full_active,2881,9141209460,1,0,0,0,0,3,643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732586,20220417,25491,3336,4,2,high_active,4503,9156857725,1,0,0,0,0,2,2156
732587,20220410,25491,6606,4,2,high_active,2881,9141209460,1,0,0,0,0,2,643
732588,20220428,25491,1799,2,6,high_active,2462,9155771466,1,0,0,0,0,2,398
732589,20220428,25491,1799,2,6,high_active,2462,9155771466,1,0,0,0,0,2,398


In [9]:
seed_tensorflow(seed=para['seed'])
"""
    uid_lay = get_layer((1,),'uid',d1 = max(action['user_id'])+1,d2 = para['embedding_dim'],trainable=True)
    videoid_lay = get_layer((1,),'video_id',d1 = max(action['video_id'])+1,d2 = int(para['embedding_dim']),trainable=True)
    duration_lay = get_layer((1,),'duration_id',d1 = max(action['duration_level'])+1,d2 = int(para['embedding_dim']),trainable=True)
    tab_lay = get_layer((1,),'tab',d1 = max(action['tab'])+1,d2 = int(para['embedding_dim']),trainable=True)
    user_active_degree_encoded = get_layer((1,),'user_active_degree_encoded',d1 = max(action['user_active_degree_encoded'])+1,d2 = int(para['embedding_dim']),trainable=True)
    author_lay = get_layer((1,),'author_id',d1 = max(action['author_id'])+1,d2 = int(para['embedding_dim']),trainable=True)
    music_lay = get_layer((1,),'music_id_encoded',d1 = max(action['music_id_encoded'])+1,d2 = int(para['embedding_dim']),trainable=True)
"""

features = ['user_id','video_id','duration_level','tab','user_active_degree_encoded','author_id','music_id_encoded','video_id',select_label]

# # short_play binary_train_label effective_play long_play complete_play
def get_input(df,is_test=False):
    X = []
    for f in features:
        X.append(df[f].values.reshape(-1,1))
    y = [df[select_label].values.reshape(-1,1)]
    return X,y

X_train,y_train = get_input(train,is_test=False)
X_valid,y_valid = get_input(valid,is_test=False)
X_test,y_test = get_input(test,is_test=True)

2024-07-28 22:04:35.019620: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:35.019816: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:35.019886: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
train_X = np.hstack(X_train)
test_X = np.hstack(X_test)
valid_X = np.hstack(X_valid)

# 打印重构的数组以验证
print(train_X)

[[    0  1399     4 ...     0  1399     0]
 [    0  6857     2 ...     1  6857     0]
 [    0  5590     4 ...     2  5590     0]
 ...
 [25491  1799     2 ...   398  1799     0]
 [25491  1799     2 ...   398  1799     0]
 [25491  2426     0 ...  1069  2426     1]]


In [11]:
train_y = y_train[0].flatten()
test_y = y_test[0].flatten()
valid_y = y_valid[0].flatten()

In [12]:
train_X.shape

(732591, 9)

In [13]:
test_y

array([0, 0, 0, ..., 0, 1, 1])

In [14]:
sparse_feats = ['user_id','video_id','duration_level','tab','user_active_degree_encoded','author_id','music_id_encoded','video_id',select_label]

In [15]:
train_sparse_x = [train[f].values for f in sparse_feats]
val_sparse_x = [valid[f].values for f in sparse_feats]
test_sparse_x = [test[f].values for f in sparse_feats]

train_label = [train[select_label].values]
val_label = [valid[select_label].values]
test_label = [test[select_label].values]


In [16]:
train_sparse_x

[array([    0,     0,     0, ..., 25491, 25491, 25491]),
 array([1399, 6857, 5590, ..., 1799, 1799, 2426]),
 array([4, 2, 4, ..., 2, 2, 0]),
 array([1, 0, 1, ..., 6, 6, 2]),
 array([3, 3, 3, ..., 2, 2, 2]),
 array([5799, 5412, 4571, ..., 2462, 2462, 1381]),
 array([   0,    1,    2, ...,  398,  398, 1069]),
 array([1399, 6857, 5590, ..., 1799, 1799, 2426]),
 array([0, 0, 0, ..., 0, 0, 1])]

In [17]:
sparse_inputs = []

input0 = tf.keras.layers.Input([1], name='user_id',dtype=tf.int32)
input1 = tf.keras.layers.Input([1], name='video_id',dtype=tf.int32)
input2 = tf.keras.layers.Input([1], name='duration_level',dtype=tf.int32)
input3 = tf.keras.layers.Input([1], name='tab',dtype=tf.int32)
input4 = tf.keras.layers.Input([1], name='user_active_degree_encoded',dtype=tf.int32)
input5 = tf.keras.layers.Input([1], name='author_id',dtype=tf.int32)
input6 = tf.keras.layers.Input([1], name='music_id_encoded',dtype=tf.int32)
input7 = tf.keras.layers.Input([1], name='video_id1',dtype=tf.int32)
input8 = tf.keras.layers.Input([1], name=select_label,dtype=tf.int32)

sparse_inputs.append(input0)
sparse_inputs.append(input1)
sparse_inputs.append(input2)

sparse_inputs.append(input3)
sparse_inputs.append(input4)
sparse_inputs.append(input5)

sparse_inputs.append(input6)
sparse_inputs.append(input7)
sparse_inputs.append(input8)

In [18]:
sparse_inputs

[<KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_id')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'video_id')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'duration_level')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'tab')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_active_degree_encoded')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'author_id')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'music_id_encoded')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'video_id1')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'effective_play')>]

In [19]:
sparse_kd_embed = []
for i, _input in enumerate(sparse_inputs):
    if i == 7:
        tf.keras.layers.Embedding(input_dim=int(feed_emb.shape[0]),
                       output_dim=int(feed_emb.shape[1]),
                       weights=[feed_emb],
                       trainable=False,
                       name='pre_embedding')
        sparse_kd_embed.append(_embed)
    elif i==8:
        continue
    else:
        f = sparse_feats[i]
        voc_size = action[f].nunique()
        _embed = tf.keras.layers.Embedding(voc_size+1, 64, embeddings_regularizer=tf.keras.regularizers.l2(0.5))(_input)
        sparse_kd_embed.append(_embed)
    


2024-07-28 22:04:35.185278: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:35.185486: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:35.185554: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [20]:
sparse_kd_embed

[<KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_1')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_2')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_3')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_4')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_5')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_6')>,
 <KerasTensor: shape=(None, 1, 64) dtype=float32 (created by layer 'embedding_6')>]

In [21]:
input_embeds = sparse_kd_embed

In [22]:
embed_map = tf.keras.layers.Concatenate(axis=1)(input_embeds) 

In [23]:

def auto_interacting(embed_map, d=6, n_attention_head=2):
    """
    实现单层 AutoInt Interacting Layer
    @param embed_map: 输入的embedding feature map, (?, n_feats, n_dim)
    @param d: Q,K,V映射后的维度
    @param n_attention_head: multi-head attention的个数
    """
    assert len(embed_map.shape) == 3, "Input embedding feature map must be 3-D tensor."
    
    k = embed_map.shape[-1]
    
    # 存储多个self-attention的结果
    attention_heads = []
    W_Q = []
    W_K = []
    W_V = []
    
    # 1.构建多个attention
    for i in range(n_attention_head):
        # 初始化W_Q, W_K, W_V
        W_Q.append(tf.Variable(tf.random.truncated_normal(shape=(k, d)), name="query_"+str(i)))  # k, d
        W_K.append(tf.Variable(tf.random.truncated_normal(shape=(k, d)), name="key_"+str(i)))  # k, d
        W_V.append(tf.Variable(tf.random.truncated_normal(shape=(k, d)), name="value_"+str(i)))  # k, d
     
    for i in range(n_attention_head):
        # 映射到d维空间
        embed_q = tf.matmul(embed_map, W_Q[i])  # ?, 39, d
        embed_k = tf.matmul(embed_map, W_K[i])  # ?, 39, d
        embed_v = tf.matmul(embed_map, W_V[i])  # ?, 39, d
    
        # 计算attention
        energy = tf.matmul(embed_q, tf.transpose(embed_k, [0, 2, 1]))  # ?, 39, 39
        attention = tf.nn.softmax(energy)  # ?, 39, 39
    
        attention_output = tf.matmul(attention, embed_v)  # ?, 39, d
        attention_heads.append(attention_output)
    
    # 2.concat multi head
    multi_attention_output = tf.keras.layers.Concatenate(axis=-1)(attention_heads)  # ?, 39, n_attention_head*d
    
    # 3.ResNet
    w_res = tf.Variable(tf.random.truncated_normal(shape=(k, d*n_attention_head)), name="w_res_"+str(i))  # k, d*n_attention_head
    output = tf.keras.layers.Activation("relu")(multi_attention_output + tf.matmul(embed_map, w_res))  # ?, 39, d*n_attention_head)
    
    return output

In [24]:
def build_autoint(x0, n_layers):
    xl = x0
    for i in range(n_layers):
        xl = auto_interacting(xl, d=6, n_attention_head=2)
    
    return xl

In [25]:
embed_map

<KerasTensor: shape=(None, 8, 64) dtype=float32 (created by layer 'concatenate')>

In [26]:
autoint_layer = build_autoint(embed_map, 3)

In [27]:
embed_map

<KerasTensor: shape=(None, 8, 64) dtype=float32 (created by layer 'concatenate')>

In [28]:
autoint_layer = tf.keras.layers.Flatten()(autoint_layer)

In [29]:
autoint_layer

<KerasTensor: shape=(None, 96) dtype=float32 (created by layer 'flatten')>

In [30]:
flattened_embed_map = tf.keras.layers.Flatten()(embed_map)
flattened_embed_map

<KerasTensor: shape=(None, 512) dtype=float32 (created by layer 'flatten_1')>

In [31]:
vec = tf.keras.layers.Dense(256,activation = 'relu',
                name='mlp_dense0')(flattened_embed_map)

vec = tf.keras.layers.Dropout(0.5)(vec)

vec = tf.keras.layers.Dense(128,
                            activation = 'relu',
                            name='mlp_dense1')(vec)

vec = tf.keras.layers.Dropout(0.5)(vec)

instance = tf.keras.layers.Dense(64,
                            activation = 'relu',
                            name='instance')(vec)

In [32]:
instance

<KerasTensor: shape=(None, 64) dtype=float32 (created by layer 'instance')>

In [33]:
final_instance = tf.keras.layers.Concatenate(axis=-1)([autoint_layer,instance]) 

In [34]:
final_instance

<KerasTensor: shape=(None, 160) dtype=float32 (created by layer 'concatenate_4')>

In [35]:
base_model = Model(sparse_inputs, final_instance)
vec = base_model.outputs[0]

final_output = tf.keras.layers.Dense(1,"sigmoid")(vec)
model = Model(inputs=base_model.inputs, outputs=final_output)

model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 user_id (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 video_id (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 duration_level (InputLayer  [(None, 1)]                  0         []                            
 )                                                                                                
                                                                                                  
 tab (InputLayer)            [(None, 1)]                  0         []                      

In [36]:
class Prototypes(tf.keras.layers.Layer):
    name = 'prototypes'
    def __init__(self,
                 k ,
                 beta1=0.0,
                 beta2=0.0,
                 beta3=0.0,
                 init_prototypes = None,
                 **kwargs):
        super(Prototypes, self).__init__(**kwargs)
        self.k = k
        self.beta1, self.beta2, self.beta3 = beta1, beta2, beta3
        # y_train[0]
        self.init_prototypes = init_prototypes
        self.binary_classifiers = []
        # 
        for _ in range(5):
            mlp = tf.keras.Sequential([
                tf.keras.layers.Dense(1, activation='sigmoid')
            ])
            mlp.build((None, 160))
            self.binary_classifiers.append(mlp)
            
    # define prototype
    def build(self, input_shape):
        self.prototypes = self.add_weight(
            name='prototypes',
            shape=(1, self.k, 160),
            initializer=tf.keras.initializers.Constant(self.init_prototypes[tf.newaxis, :, :]),
            trainable=True
        )
        super(Prototypes, self).build(input_shape)

    
    def orthogonality_loss(self,prototypes, k):
        # 确保传入的 k 是偶数
        assert k % 2 == 0, "k must be even."
        D = k // 2

        prototypes = tf.reshape(prototypes, [k, 160])
        prototypes = tf.nn.l2_normalize(prototypes, axis=1)

        cosine_similarity_matrix = tf.matmul(prototypes, prototypes, transpose_b=True)

        upper_triangular_part = tf.linalg.band_part(cosine_similarity_matrix, 0, -1)
        upper_triangular_part -= tf.linalg.band_part(cosine_similarity_matrix, 0, 0)  # 去掉对角线

        loss = tf.reduce_sum(tf.square(upper_triangular_part))
        normalization_factor = D * (2 * D - 1)
        loss /= normalization_factor

        return loss

    def contrastive_loss(self, vec, labels, duration):
        # Normalize the vectors
        vec_norm = tf.nn.l2_normalize(vec, axis=1)
        #print("vec_norm",vec_norm)
        sim_matrix = tf.matmul(vec_norm, vec_norm, transpose_b=True)
        #print("sim_matrix",sim_matrix)
        # Create masks
        label_eq = tf.equal(labels, tf.transpose(labels))
        #print("label_eq",label_eq)
        duration_eq = tf.equal(duration, tf.transpose(duration))
        #print("duration_eq",duration_eq)
        mask = tf.logical_and(label_eq, duration_eq)
        mask = tf.cast(mask, dtype=tf.float32)
        #print("mask",mask)

        # Compute the softmax denominator
        exp_sim = tf.exp(sim_matrix)
        total_sum = tf.reduce_sum(exp_sim)
        #print("total_sum",total_sum)
        normal_sim = exp_sim / total_sum
        #print("normal_sim",normal_sim)

        log_normal_sim = tf.math.log(normal_sim)
        weighted_log_prob = mask * log_normal_sim

        num_ones = tf.reduce_sum(mask)

        loss = -tf.reduce_sum(weighted_log_prob)
        return loss/num_ones

    def assign_loss(self,output, vec):


        output_norm = tf.nn.l2_normalize(output, axis=1)
        vec_norm = tf.nn.l2_normalize(vec, axis=1)

        cosine_similarity = tf.reduce_sum(tf.multiply(output_norm, vec_norm), axis=1)
        loss = 1 - tf.reduce_mean(cosine_similarity)
        return loss
    def get_alpha(self,n_batch,duration_one_hot_expanded,alpha_product):
        """
        alpha_product : (n,10,1)
        duration_one_hot_expanded: (n,5,1)
        """
        expanded_alpha = tf.reshape(alpha_product, [n_batch, 5, 2, 1])
        masked = expanded_alpha * tf.cast(duration_one_hot_expanded, tf.float32)[:, :, :, tf.newaxis]
        # n,2,1
        alpha_para = tf.reduce_sum(masked, axis=1)
        alpha_neg = alpha_para[:, 0, :]  # 第一个元素，保留最后一个轴
        alpha_pos = alpha_para[:, 1, :]  # 第二个元素，保留最后一个轴
        return alpha_neg,alpha_pos
    def mse_loss(self,vec,label):
        vec = tf.convert_to_tensor(vec, dtype=tf.float32)

        label = tf.convert_to_tensor(label, dtype=tf.float32)

        squeezed_label = tf.squeeze(label)

        #tf.print(label, summarize=-1)
        positive_indices = tf.where(squeezed_label == 1)
        negative_indices = tf.where(squeezed_label == 0)

        positive_samples = tf.gather(vec, positive_indices)
        negative_samples = tf.gather(vec, negative_indices)
        positive_samples = tf.squeeze(positive_samples,axis=1)
        negative_samples = tf.squeeze(negative_samples,axis=1)
        positive_loss = tf.reduce_mean(tf.square(positive_samples - self.pos_mean_prototype))
        negative_loss = tf.reduce_mean(tf.square(negative_samples - self.neg_mean_prototype))

        total_loss = positive_loss * 0.7 + negative_loss * 0.3

        return total_loss

    def binary_loss(self,y_true, y_pred):
        bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
        loss = bce(y_true, y_pred)
        return loss     
    
    def create_mask(self,duration, batch_size, num_classes=40, bucket_size=8):
        start_indices = duration * bucket_size
        masks = []
        for i in range(batch_size):
            mask = tf.concat([
                tf.zeros((start_indices[i, 0], 1), dtype=tf.float32),
                tf.ones((bucket_size, 1), dtype=tf.float32),
                tf.zeros((num_classes - start_indices[i, 0] - bucket_size, 1), dtype=tf.float32)
            ], axis=0)
            masks.append(mask)
        return tf.stack(masks)
    def binary_crossentropy_manual(self, y_true, y_pred,para):
        y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

        # 防止 log(0) 的情况发生
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)

        loss = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)

        weights = tf.where(y_true == 1, para, 1.0)
        weighted_loss = loss * weights

        positive_loss = tf.boolean_mask(weighted_loss, y_true == 1)
        negative_loss = tf.boolean_mask(weighted_loss, y_true == 0)
        #total_positive_loss = tf.reduce_sum(positive_loss)
        #total_negative_loss = tf.reduce_sum(negative_loss)

        return tf.reduce_mean(weighted_loss)
    def prototype_loss(self,prototypes_output):
        bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
        labels = tf.constant([[0], [1], [0], [1], [0], [1], [0], [1], [0], [1]], dtype=tf.float32)
        loss = bce(labels, prototypes_output)
        return loss
    
    def call(self, x, training=None):

        vec,duration,label = x
        a = tf.expand_dims(vec, -2)

        b = self.prototypes
        n_batch = tf.shape(vec)[0]

        dot_product = tf.multiply(a, b)

        norm_a = tf.norm(a, axis=-1, keepdims=True)  # 保持维度，使得维度是 [batch_size, 1, 1]

        norm_b = tf.norm(b, axis=-1, keepdims=True)  # 保持维度，使得维度是 [1, k, 1]

        # (batch_size, k,1)
        sum_product = tf.reduce_sum(dot_product, axis=-1, keepdims=True)
        cos_similarity = sum_product / (norm_a * norm_b)
        
        #tf.print(cos_similarity.shape)
        """
        tf.print(vec,summarize=-1)
        tf.print(self.prototypes,summarize=-1)
        tf.print("Input duration")
        tf.print(duration.shape)
        tf.print("Input label")
        tf.print(label.shape)
        #tf.print(vec)
        tf.print(duration, summarize=-1)
        tf.print(label, summarize=-1)

        tf.print(dot_product.shape)
        tf.print(dot_product)

        tf.print(sum_product.shape)
        tf.print(sum_product,summarize=-1)

        tf.print(norm_a.shape)
        tf.print(norm_a)

        tf.print(norm_b.shape)
        tf.print(norm_b)

        tf.print(cos_similarity, summarize=-1)
        """
        # （batch_size, k//2, 2, 1)
        reshaped_sum_product = tf.reshape(cos_similarity, (n_batch, self.k // 2, 2))

        temperature = 0.05

        scaled_logits = reshaped_sum_product / temperature

        softmaxed_sum_product = tf.nn.softmax(scaled_logits, axis=2)

        # (batch_size, k,1)
        alpha_product = tf.reshape(softmaxed_sum_product, (n_batch, self.k, 1))
        #tf.print(alpha_product,summarize=-1)
        """
        tf.print(alpha_product.shape)
        tf.print(alpha_product)
        """
        #self.new = alpha_product
        # n_batch, self.k, 64
        # b:(1, k, 64)
        alpha_product_expanded = tf.broadcast_to(alpha_product, [n_batch, self.k, 160])

        # calculate
        weighted_products = alpha_product_expanded * b

        # add up
        reshaped_weighted_products = tf.reshape(weighted_products, [n_batch, self.k//2, 2, 160])

        # (batchsize,5,64)
        pre_output = tf.reduce_sum(reshaped_weighted_products, axis=2)  # 对第三个维度求和

        # 1 -> [0,1,0,0,0]
        duration_one_hot = tf.one_hot(duration, depth=5)
        duration_one_hot_expanded = tf.expand_dims(duration_one_hot, -1)
        #(batch_size, 5, 1)
        duration_one_hot_expanded = tf.squeeze(duration_one_hot_expanded, axis=[1])

        masked_output = pre_output * duration_one_hot_expanded
        # output (batchsize,64)
        output = tf.reduce_sum(masked_output, axis=1)


        # get_alpha
        alpha_neg, alpha_pos = self.get_alpha(n_batch,duration_one_hot_expanded,alpha_product)
        #tf.print(alpha_pos,summarize = -1)
        #tf.print(alpha_neg,summarize = -1)
        #tf.print(alpha_pos,summarize   =-1)
        #tf.print(label, summarize=-1)
        classifier_outputs = []


        
        for i in range(5):
            classifier_output = self.binary_classifiers[i](tf.squeeze(self.prototypes,axis=0))
             # (batch_size, 1, 1)#
            #classifier_output += i
            classifier_outputs.append(classifier_output)
        
        #tf.print(classifier_outputs,summarize = -1)
        #tf.print(classifier_outputs.shape,summarize = -1)
        indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]  
        prototype_output_elements = []

        for i, idx in enumerate(indices):
            tensor_index = i // 2  
            element = tf.gather(classifier_outputs[tensor_index], idx, axis=0) 
            prototype_output_elements.append(element)

        prototype_output = tf.concat(prototype_output_elements, axis=0)
        prototype_output = tf.expand_dims(prototype_output,axis=1)
        #(1,10)
  
        positive_indices = 2 * duration + 1
        negative_indices = 2 * duration

        positive_indices = tf.clip_by_value(positive_indices, 0, prototype_output.shape[0] - 1)
        negative_indices = tf.clip_by_value(negative_indices, 0, prototype_output.shape[0] - 1)
        #tf.print(alpha_pos,summarize = -1)
        # 提取对应的 prototype_output 值
        positive_values = tf.gather(prototype_output, positive_indices)
        negative_values = tf.gather(prototype_output, negative_indices)
        #positive_values = tf.squeeze(positive_values,axis=0)
        #negative_values = tf.squeeze(negative_values,axis=0)
        output = alpha_pos * tf.squeeze(positive_values,axis=-1) 
        assign_loss = self.assign_loss(output,vec)

        #binary_loss = self.binary_crossentropy_manual(label,output,1.0)
        #self.add_loss(1.0 * binary_loss)
        # 5
        assign = self.binary_loss(label,alpha_pos)
        self.add_loss(0.1 * assign)

        ortho_loss = self.orthogonality_loss(self.prototypes,self.k)
        self.add_loss(0.3 * ortho_loss)

        contrastive_loss = self.contrastive_loss(vec,label,duration)
        self.add_loss(0.12 * contrastive_loss)

        proto_loss = self.prototype_loss(prototype_output)
        self.add_loss(0.1 * proto_loss)


        return output

In [37]:
proto_tensor = tf.random.normal([10, 160], mean=0.0, stddev=1.0, dtype=tf.float32)

In [38]:
from tensorflow.keras.models import Model
seed_tensorflow(seed=para['seed'])
#backbone.load_weights("./model/kuairand_CPF/mlp_effect_backbone.h5")

duration_id_input_output = base_model.get_layer("duration_level").output
labels_output = base_model.get_layer(select_label).output

flag = 1
if flag:
    prototypes_layer = Prototypes(k=10,init_prototypes = proto_tensor
                                  )

    output = prototypes_layer([base_model.outputs[0],duration_id_input_output,labels_output])
    #output.shape
else:
    output = tf.keras.layers.Dense(1, activation='sigmoid')(base_model.outputs[0])


model = tf.keras.Model(inputs=base_model.inputs, outputs=[output])

seed_tensorflow(seed=para['seed'])

adam=tf.keras.optimizers.Adam(learning_rate=para['lr'])
model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=['AUC'])




2024-07-28 22:04:35.796312: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:35.796661: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:35.796983: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

2024-07-28 22:04:36.201898: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:36.202166: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:36.202254: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [39]:
seed_tensorflow(seed=para['seed'])

es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # 监控验证集损失
    mode='min',          # 最小化验证集损失
    patience=para['callback']['patience']  # 等待的轮数
)

checkpoint_auc = tf.keras.callbacks.ModelCheckpoint(
    filepath=para['callback']['base_model'],  # 保存模型的路径
    monitor='val_auc',  # 监控验证集 AUC
    mode='max',          # 保存验证 AUC 最大的模型
    save_weights_only=True,  # 只保存模型权重
    save_best_only=True  # 只保存最佳模型
)


hist = model.fit(train_sparse_x,
                 train_label,
                 epochs=200,
                 batch_size=512,
                 shuffle=True,
                 verbose=para['verbose'],
                 callbacks=[checkpoint_auc,es_callback],
                 validation_data=(val_sparse_x,val_label))

Epoch 1/200


2024-07-28 22:04:36.220190: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:36.220427: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:04:36.220531: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

1431/1431 [==============================] - 15s 8ms/step - loss: 796.9949 - auc: 0.6070 - val_loss: 488.1787 - val_auc: 0.6536
Epoch 2/200
1431/1431 [==============================] - 11s 8ms/step - loss: 306.5351 - auc: 0.6785 - val_loss: 168.6973 - val_auc: 0.7013
Epoch 3/200
1431/1431 [==============================] - 11s 7ms/step - loss: 94.6712 - auc: 0.7110 - val_loss: 42.5994 - val_auc: 0.7245
Epoch 4/200
1431/1431 [==============================] - 11s 8ms/step - loss: 20.4139 - auc: 0.7293 - val_loss: 7.1669 - val_auc: 0.7421
Epoch 5/200
1431/1431 [==============================] - 11s 7ms/step - loss: 3.7320 - auc: 0.7428 - val_loss: 2.2403 - val_auc: 0.7536
Epoch 6/200
1431/1431 [==============================] - 11s 7ms/step - loss: 2.1131 - auc: 0.7511 - val_loss: 2.0588 - val_auc: 0.7583
Epoch 7/200
1431/1431 [==============================] - 11s 7ms/step - loss: 2.0645 - auc: 0.7563 - val_loss: 2.0444 - val_auc: 0.7620
Epoch 8/200
1431/1431 [==========================

In [40]:

# infer
"""
0.78959733
0.46864867
"""

seed_tensorflow(seed=para['seed'])
model.load_weights(para['callback']['base_model'])
y_test_pred = model.predict(X_test,batch_size=10000)
auc_metric = tf.keras.metrics.AUC()
auc_metric.update_state(y_test[0], y_test_pred)
auc = auc_metric.result().numpy()

logloss_metric = tf.keras.metrics.BinaryCrossentropy()

logloss_metric.update_state(y_test[0], y_test_pred)
log_loss = logloss_metric.result().numpy()
print('auc:',auc)
print('logloss:',log_loss)

2024-07-28 22:16:35.217087: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:16:35.217410: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-28 22:16:35.217495: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

26/26 [==============================] - 0s 6ms/step
auc: 0.78792197
logloss: 0.47106457
